In [2]:
import os
import dotenv

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.core import Settings
import os

llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)
Settings.embed_model = embed_model

In [4]:
import phoenix as px
px.launch_app()


/home/vscode/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1765898159.985226   15280 chttp2_server.cc:1682] UNKNOWN:No address added out of total 1 resolved for '[::]:4317' {created_time:"2025-12-16T15:15:59.985221659+00:00", children:[UNKNOWN:Failed to add any wildcard listeners {created_time:"2025-12-16T15:15:59.985208451+00:00", children:[UNKNOWN:Unable to configure socket {created_time:"2025-12-16T15:15:59.979242433+00:00", fd:81, children:[UNKNOWN:bind: Address already in use (98) {created_time:"2025-12-16T15:15:59.979214132+00:00"}]}, UNKNOWN:Unable to configure socket {created_time:"2025-12-16T15:15:59.9852055+00:00", fd:81, children:[UNKNOWN:bind: Address already in use (98) {created_time:"2025-12-16T15:15:59.985190373+00:00"}]}]}]}
ERROR:    Traceback (most recent call last

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [6]:
documents = SimpleDirectoryReader("../Big Star Collectibles").load_data()

index_1 = VectorStoreIndex.from_documents(documents, insert_batch_size=150)
index_1.storage_context.persist(persist_dir="../local_index_1")

In [7]:
# one example solution
Settings.chunk_size = 500
Settings.chunk_overlap = 100
index_2 = VectorStoreIndex.from_documents(documents, insert_batch_size=150)
index_2.storage_context.persist(persist_dir="../local_index_2")

In [8]:
# solution
query_engine = index_2.as_query_engine(
  llm=llm
)
query_engine.query("When was Big Star Collectibles Started?")

Response(response='Big Star Collectibles was officially launched in 2014.', source_nodes=[NodeWithScore(node=TextNode(id_='38e59101-9f11-440c-8fdb-0c25b315acb0', embedding=None, metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapter_2/../Big Star Collectibles/Our Story.txt', 'file_name': 'Our Story.txt', 'file_type': 'text/plain', 'file_size': 877, 'creation_date': '2025-12-16', 'last_modified_date': '2025-12-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b3460fcd-d6fc-4bc1-b185-528226abd351', node_type='4', metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapte

In [9]:
query_engine.query("Who started Big Star Collectibles?")

Response(response='Big Star Collectibles was started by Saura Chen.', source_nodes=[NodeWithScore(node=TextNode(id_='38e59101-9f11-440c-8fdb-0c25b315acb0', embedding=None, metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapter_2/../Big Star Collectibles/Our Story.txt', 'file_name': 'Our Story.txt', 'file_type': 'text/plain', 'file_size': 877, 'creation_date': '2025-12-16', 'last_modified_date': '2025-12-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b3460fcd-d6fc-4bc1-b185-528226abd351', node_type='4', metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapter_2/..

In [10]:
query_engine.query("Please describe your apron product")

Response(response='The eco-friendly apron is made from a blend of 55% organic cotton and 45% recycled polyester. It features a design showcasing your favorite Big Star Collectibles artwork, which is screen-printed using PVC- and phthalate-free inks. The apron measures 24 inches wide by 30 inches long and is easily adjustable around the neck and waist with one continuous strap. For care, it can be machine-washed warm and tumble-dried on low.', source_nodes=[NodeWithScore(node=TextNode(id_='2bff8c15-e2ba-4289-9732-f12dd3e88439', embedding=None, metadata={'file_path': '/workspaces/introduction-to-retrieval-augmented-generation-rag-with-hands-on-projects-2023442/chapter_2/../Big Star Collectibles/Product Descriptions.txt', 'file_name': 'Product Descriptions.txt', 'file_type': 'text/plain', 'file_size': 3971, 'creation_date': '2025-12-16', 'last_modified_date': '2025-12-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_ac